**Inputs**
* ERA-5 data
* Hazard definition(s)

**Outputs**
* ImageCollection(s) containing hazard magnitudes for each year-model pair
* Stored in users/tedwongwri/dataportal/hist-magnitudes/HAZARDNAME
* **You must manually create the empty ImageCollection** in the dataportal/hist-magnitudes directory before trying to export results

In [1]:
import ee
ee.Authenticate()

Enter verification code: 4/1AX4XfWj8okbkYk_d-EUkUwvNhhNdTRg1pbyT0PU4d0ObFgZEdgK3fPZ4DTs

Successfully saved authorization token.


In [2]:
ee.Initialize()

In [15]:
from datetime import date, timedelta

In [52]:
# Define hazards here

def dryspells(yeardata):
    dry_days = yeardata.map(lambda d: d.eq(0))
    return process_runs(dry_days, 5, 'count')


def maxdryspell(yeardata):
    # Climdex CDD: Let RR_ij be the daily precipitation amount on day i in period j. Count the largest number of consecutive days where RR_ij < 1mm
    dry_days = yeardata.map(lambda d: d.multiply(86400).eq(0).multiply(1))
    return process_runs(dry_days, 1, 'max')

def ehe(yeardata):
    # Note: all in Kelvins
    over_year = yeardata.map(lambda x: x.gt(TASMAX_99).multiply(1))
    return over_year.sum()

def mtt35(yeardata):
    over_year = yeardata.map(lambda x: x.gt(35 + 273.15).multiply(1))
    return over_year.sum()

susc = ee.Image("users/tedwongwri/dataportal/landslide/susc")
ari95 = ee.Image("users/tedwongwri/dataportal/landslide/ARI95")
def modlandslide(yeardata):
# Returns number of days in year with moderate or high landslide risk
# antecedent rainfall index from https://agupubs.onlinelibrary.wiley.com/doi/full/10.1002/2017EF000715
    start_date, end_date = date(year, 1, 1), date(year, 12, 31)
    modeldata = era5_pr
    modrisk_list = ee.List([]) # holds results
    precip_list = ee.List([ee.Image(0)]) # holds input precip for current date and six previous; zero image is dummy, will be removed immediately
    for t in range(6):  # Create list with seven days of data, with most recent last
        tt = 6 - t
        tdate = start_date - timedelta(days=tt)
        modelscenario_data = modeldata
        precip_list = precip_list.add(modelscenario_data.filterMetadata('system:index', 'equals', tdate.isoformat().replace('-', '')).first().multiply(86400))
    for datediff in range((end_date - start_date).days):
        focal_date = start_date + timedelta(days = datediff)
        modelscenario_data = modeldata
        precip_list = precip_list.slice(1, 7)  #Every day remove one, add one
        precip_list = precip_list.add(modelscenario_data.filterMetadata('system:index', 'equals', focal_date.isoformat().replace('-', '')).first().multiply(86400))
        numerator = ee.Image.constant(0)
        denominator = ee.Image.constant(0)
        for t in range(7):
            tdate = focal_date - timedelta(days=t)
            precip = ee.Image(precip_list.get(-(t + 1)))
            w = 1.0 / ((t + 1)**2)
            numerator = numerator.add(precip.multiply(w))
            denominator = denominator.add(w)
        modrisk_list = modrisk_list.add((numerator.divide(denominator).gt(ari95).multiply(1)).multiply(susc.gt(2).multiply(1)))
    return ee.ImageCollection(modrisk_list).sum()

def highlandslide(yeardata):
# Returns number of days in year with moderate or high landslide risk
# antecedent rainfall index from https://agupubs.onlinelibrary.wiley.com/doi/full/10.1002/2017EF000715
    start_date, end_date = date(year, 1, 1), date(year, 12, 31)
    modeldata = era5_pr
    highrisk_list = ee.List([]) # holds results
    precip_list = ee.List([ee.Image(0)]) # holds input precip for current date and six previous; zero image is dummy, will be removed immediately
    for t in range(6):  # Create list with seven days of data, with most recent last
        tt = 6 - t
        tdate = start_date - timedelta(days=tt)
        modelscenario_data = modeldata
        precip_list = precip_list.add(modelscenario_data.filterMetadata('system:index', 'equals', tdate.isoformat().replace('-', '')).first().multiply(86400))
    for datediff in range((end_date - start_date).days):
        focal_date = start_date + timedelta(days = datediff)
        modelscenario_data = modeldata
        precip_list = precip_list.slice(1, 7)  #Every day remove one, add one
        precip_list = precip_list.add(modelscenario_data.filterMetadata('system:index', 'equals', focal_date.isoformat().replace('-', '')).first().multiply(86400))
        numerator = ee.Image.constant(0)
        denominator = ee.Image.constant(0)
        for t in range(7):
            tdate = focal_date - timedelta(days=t)
            precip = ee.Image(precip_list.get(-(t + 1)))
            w = 1.0 / ((t + 1)**2)
            numerator = numerator.add(precip.multiply(w))
            denominator = denominator.add(w)
        highrisk_list = highrisk_list.add((numerator.divide(denominator).gt(ari95).multiply(1)).multiply(susc.gt(4).multiply(1)))
    return ee.ImageCollection(highrisk_list).sum()


hazards = [
            {'definition': ehe,
             'variable': 'tasmax'
            },
            {'definition': mtt35,
             'variable': 'tasmax'
            },
            {'definition': modlandslide,
             'variable': 'pr'
            },
            {'definition': highlandslide,
             'variable': 'pr'
            }
]

In [9]:
def process_runs(imgs, runLength, resultType): 
# from Logan Byers
    def doOne(img, data):
   #data = ee.Image(data);

        dataDict = ee.Dictionary(data)

        previousThresholdImage = ee.Image(dataDict.get('previousThresholdImage'))
        currentStreakImage = ee.Image(dataDict.get('currentStreakImage')).uint16()
        streakCountImage = ee.Image(dataDict.get('streakCountImage')).uint16()
        longestStreakImage = ee.Image(dataDict.get('longestStreakImage')).uint16()
        streakAccumulation = ee.Image(dataDict.get('streakAccumulation')).uint16()
        numRemaining = ee.Number(dataDict.get('numRemaining'))

        # WHERE yesterday AND today : 1, else 0
        #continueStreakImage = previousThresholdImage.and(ee.Image(img))
        continueStreakImage = previousThresholdImage.multiply(ee.Image(img)).multiply(ee.Image(numRemaining).gt(1).multiply(1))

        # WHERE NOT on streak :  yesterday streak length, else 0
        #streakEndedImage = currentStreakImage.multiply(currentStreakImage.and(continueStreakImage.not()))
        streakEndedImage = currentStreakImage.multiply(currentStreakImage.multiply(continueStreakImage.multiply(-1).add(1)))

        # WHERE NOT on streak AND yesterday streak length > length threshold : 1, else 0
        endedStreakExceedsLengthImage = currentStreakImage.multiply(streakEndedImage).gte(runLength)

        # update the state
        accumulator = ee.Dictionary.fromLists([
            'previousThresholdImage',
            'currentStreakImage',
            'streakCountImage',
            'longestStreakImage',
            'streakAccumulation',
            'numRemaining'
          ], [
            # previousThresholdImage --> today's image
            ee.Image(img),
            # currentStreakImage --> today's image PLUS yesterday's streak (where continuing)
            currentStreakImage
              .multiply(continueStreakImage).add(ee.Image(img)),
            # streakCountImage --> PLUS 1 where long streak ended today
            streakCountImage.add(endedStreakExceedsLengthImage),
            # longestStreakImage --> larger of prev and current value
            longestStreakImage.max(currentStreakImage.multiply(continueStreakImage).add(ee.Image(img))),
            # streakAccumulation --> yesterday's accum plus current 1/0, if in streak
            streakAccumulation.add(ee.Image(img).multiply(continueStreakImage)),
            # numRemaining --> minus 1
            numRemaining.add(-1)
          ]
        )

        return accumulator
  
    resultImageName = {
        'count': 'streakCountImage',
        'max': 'longestStreakImage',
        'accum': 'streakAccumulation'
    }
  
    streakData = imgs.iterate(
    # iterate over each image in the ImageCollection
    #   accumulate a stateful Dictionary of images
    
        doOne, ee.Dictionary.fromLists([
          'previousThresholdImage', 
          'currentStreakImage',
          'streakCountImage',
          'longestStreakImage',
          'streakAccumulation',
          'numRemaining'
        ], [
          ee.Image.constant(1),
          ee.Image.constant(0).uint16(),
          ee.Image.constant(0).uint16(),
          ee.Image.constant(0).uint16(),
          ee.Image.constant(0).uint16(),
          imgs.size().add(-1)
        ]
        )
    
    )
    return ee.Image(ee.Dictionary(streakData).get(resultImageName[resultType]))

In [10]:
era5_tasmax = ee.ImageCollection("ECMWF/ERA5/DAILY").select('maximum_2m_air_temperature')
era5_tasmin = ee.ImageCollection("ECMWF/ERA5/DAILY").select('minimum_2m_air_temperature')
era5_pr = ee.ImageCollection("ECMWF/ERA5/DAILY").select('total_precipitation')
PR_99 = ee.ImageCollection('users/tedwongwri/dataportal/percentiles/pr_99').mean()
TASMAX_99 = ee.ImageCollection('users/tedwongwri/dataportal/percentiles/tasmax_99').mean()
TASMIN_01 = ee.ImageCollection('users/tedwongwri/dataportal/percentiles/tasmin_01').mean()

# ERA5 reports precip in m/d, while NEX-GDDP reports it as kg m^-2 s^-1
# 1kg water spread out over 1m is 1mm thick
# 1d = 86400s
#           1m/d  *  1000mm/m  *  1d/86400s  *  1kg/(mm * m^2) = 0.011574074074074073 m^-2 s^-1
# So to express ERA5 precip in NEx units:
era5_pr = era5_pr.map(lambda i: i.multiply(0.011574074074074073))

In [11]:
WHOLE_GLOBE = ee.Geometry.Rectangle([-179.999, -90, 180, 90], 'EPSG:4326', False)
START_YEAR = 1980
END_YEAR = 2019

In [51]:
for hazard in hazards[3:]:
    for year in range(START_YEAR, END_YEAR + 1):
        dataset = {'tasmax': era5_tasmax, 'tasmin': era5_tasmin, 'pr': era5_pr}[hazard['variable']]
        data = dataset.filterDate(str(year) + '-01-01', str(year) + '-12-31')
        img = hazard['definition'](data)
        task = ee.batch.Export.image.toAsset(**{
          'image': img.rename(hazard['definition'].__name__).set('model', 'era5').set('year', year),
          'description': '{0}_{1}_{2}'.format(hazard['definition'].__name__, str(year), 'ERA5'),
          'assetId': 'users/tedwongwri/dataportal/hist-magnitudes/{0}/{1}_{2}'.format(hazard['definition'].__name__, str(year), 'ERA5'),
          'region': WHOLE_GLOBE,
          'crs': 'EPSG:4326',
          'dimensions': '1440x720'
        })
        task.start()